In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras
import tensorflow as tf
from nltk.corpus import stopwords
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.stem import PorterStemmer
import string
data = pd.read_json("./Sarcasm_Headlines_Dataset_v2.json",lines=True)

In [4]:
import nltk
nltk.download('stopwords')

stwrds = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# remove duplicate headlines
data=data.drop(data[data['headline'].duplicated()].index,axis=0)
sarc_cnt = len(data.query('is_sarcastic==1'))
non_sarc_cnt = len(data.query('is_sarcastic==0'))

# Summary of sarcastic lines
print(f'There are {sarc_cnt} sarcastic headlines and {non_sarc_cnt} non-sarcastic headlines')
# import stopwords from nltk
stwrds = set(stopwords.words('english'))
ps = PorterStemmer()
# method to clean a given headline by lowercasing the string, removing spaces, and removing stopwords
def clean_headlines(headline):
    headline = headline.lower()
    headline_split = headline.split()
    cleaned_headline = []
    for word in headline_split:
        if word not in stwrds and word not in string.punctuation:
            cleaned_headline.append(ps.stem(word))
    cleaned_line = " ".join(cleaned_headline)
    return cleaned_line
data['cleaned'] = data['headline'].apply(clean_headlines)

There are 13552 sarcastic headlines and 14951 non-sarcastic headlines


In [6]:
labels=data.is_sarcastic 
features=data.cleaned

In [7]:
feature_train,feature_test, labels_train,labels_test = train_test_split(features,labels ,test_size =0.2)

In [8]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(feature_train)

vocab_size = len(tokenizer.word_index)
print(vocab_size)

train_sequences = tokenizer.texts_to_sequences(feature_train)
test_sequences = tokenizer.texts_to_sequences(feature_test)

21892


In [9]:
maxlen=max([len(i) for i in train_sequences])

In [10]:
train_padded = pad_sequences(train_sequences, maxlen=maxlen,  padding='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen,  padding='post')

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
# from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [12]:
neigh = KNeighborsClassifier(n_neighbors=11,metric="euclidean")
neigh.fit(train_padded,labels_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=11)

In [13]:
print(classification_report(labels_test, neigh.predict(test_padded)))

              precision    recall  f1-score   support

           0       0.56      0.64      0.60      3014
           1       0.52      0.44      0.47      2687

    accuracy                           0.54      5701
   macro avg       0.54      0.54      0.54      5701
weighted avg       0.54      0.54      0.54      5701



In [14]:
from tqdm import tqdm

In [15]:
for i in tqdm (range (5,20,2)):
    # if i in[0,1,10]: continue
    neigh = KNeighborsClassifier(n_neighbors=i,metric="euclidean")
    neigh.fit(train_padded,labels_train)
    print("Number of neighbors: ", i)
    print(classification_report(labels_test, neigh.predict(test_padded)))

  0%|          | 0/8 [00:00<?, ?it/s]

Number of neighbors:  5


 12%|█▎        | 1/8 [00:02<00:19,  2.77s/it]

              precision    recall  f1-score   support

           0       0.56      0.61      0.59      3014
           1       0.52      0.46      0.49      2687

    accuracy                           0.54      5701
   macro avg       0.54      0.54      0.54      5701
weighted avg       0.54      0.54      0.54      5701

Number of neighbors:  7


 25%|██▌       | 2/8 [00:07<00:23,  3.99s/it]

...

              precision    recall  f1-score   support

           0       0.56      0.66      0.61      3014
           1       0.52      0.42      0.47      2687

    accuracy                           0.55      5701
   macro avg       0.54      0.54      0.54      5701
weighted avg       0.54      0.55      0.54      5701

Number of neighbors:  17


 88%|████████▊ | 7/8 [00:24<00:03,  3.11s/it]

              precision    recall  f1-score   support

           0       0.56      0.66      0.61      3014
           1       0.52      0.42      0.46      2687

    accuracy                           0.55      5701
   macro avg       0.54      0.54      0.54      5701
weighted avg       0.54      0.55      0.54      5701

Number of neighbors:  19


100%|██████████| 8/8 [00:26<00:00,  3.37s/it]

              precision    recall  f1-score   support

           0       0.56      0.67      0.61      3014
           1       0.53      0.42      0.47      2687

    accuracy                           0.55      5701
   macro avg       0.55      0.54      0.54      5701
weighted avg       0.55      0.55      0.54      5701

